In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install -qqq openai
!pip install -Uqqq chromadb

In [ ]:
import os
import textwrap

import langchain
import chromadb
import transformers
import openai
import torch

from transformers import AutoTokenizer
from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
!huggingface-cli login

In [ ]:
os.environ["OPENAI_API_KEY"] = "INSERT_YOUR_API_KEY"

In [ ]:
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
      "text-generation", 
      model=model,
      tokenizer=tokenizer,
      torch_dtype=torch.bfloat16,
      trust_remote_code=True,
      device_map="auto",
      max_length=1000,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
# Load documents locally as CSV
loader = CSVLoader('/content/sample_data/Frangrances-Dataset.csv')
docs = loader.load()
docs[0]

In [ ]:
# Split document into text chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs)

In [ ]:
# Initialize the open-source embedding function, default: text-embedding-ada-002
embedding_function = OpenAIEmbeddings()

In [ ]:
# Load it into ChromaDB
db = Chroma.from_documents(docs, embedding_function)

In [ ]:
#Design Prompt Template
template = """You are a customer service chatbot for an online perfume company called Fragrances International.

{context}

Answer the customer's questions only using the source data provided. If you are unsure, say "I don't know, please call our customer support". Use engaging, courteous, and professional language similar to a customer representative.
Keep your answers concise.

Question:

Answer: """

In [ ]:
#Intiliaze prompt using prompt template via langchain
prompt = PromptTemplate(template=template, input_variables=["context"])
print(
    prompt.format(
        context = "A customer is on the perfume company website and wants to chat with the website chatbot."
    )
)

In [ ]:
#Chain to have all components together and query the LLM
chain_type_kwargs = {"prompt": prompt}

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
)

In [ ]:
# Formatted printing
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=80)))

In [ ]:
#Running chain through LLM with query
query = "What types of perfumes do you sell?"
response = chain.run(query)
print_response(response)